<a href="https://colab.research.google.com/github/x1001000/raspberrypi3-yamnet-sed/blob/main/colab_notebooks/Transfer_Learning_From_YAMNet_to_YAMNet-tw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# runtime reset

## install

In [ ]:
!pip install soundfile
!pip install git+https://github.com/nficano/pytube
!pip install pydub

  Cloning https://github.com/nficano/pytube to /tmp/pip-req-build-5i1zoiy2
  Running command git clone -q https://github.com/nficano/pytube /tmp/pip-req-build-5i1zoiy2
  Created wheel for pytube: filename=pytube-10.4.1-cp36-none-any.whl size=43322 sha256=104b8590cc8ef8770f4ff04788c1086fcf75dbfcea451fdc4c7c6a75c4094808
  Stored in directory: /tmp/pip-ephem-wheel-cache-vtue6l7t/wheels/44/da/40/3b5e03abe33a91895343814fb44b309512375408f4a909555b
Successfully built pytube


## download YAMNet (15M bytes)

In [ ]:
# !curl -O https://storage.googleapis.com/audioset/yamnet.h5
# !git clone https://github.com/tensorflow/models
# !cp models/research/audioset/yamnet/* .
!git clone https://github.com/x1001000/raspberrypi3-yamnet-sed
!cp raspberrypi3-yamnet-sed/yamnet/* .

Cloning into 'raspberrypi3-yamnet-sed'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 232 (delta 133), reused 89 (delta 41), pack-reused 0
Receiving objects: 100% (232/232), 23.34 MiB | 20.04 MiB/s, done.
Resolving deltas: 100% (133/133), done.


# runtime restart

## import

In [ ]:
import numpy as np
import resampy
import soundfile as sf
import tensorflow as tf

import params as yamnet_params
import yamnet as yamnet_model

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

from pytube import Playlist, YouTube
from pydub import AudioSegment

from IPython.display import display, Audio
from time import sleep

## load YAMNet (3.7M params)

In [ ]:
params = yamnet_params.Params()
yamnet = yamnet_model.yamnet_frames_model(params)
yamnet.load_weights('yamnet.h5')
yamnet_classes = yamnet_model.class_names('yamnet_class_map_zh-tw.csv')

yamnet.summary()

Model: "yamnet_frames"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
tf.compat.v1.shape (TFOpLambda) (1,)                 0           input_1[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici ()                   0           tf.compat.v1.shape[0][0]         
__________________________________________________________________________________________________
tf.math.maximum_1 (TFOpLambda)  ()                   0           tf.__operators__.getitem[0][0]   
______________________________________________________________________________________

## def data_from(YouTube)

In [ ]:
def data_from(url, begin=1, end=None):
    waveforms = []
    titles = []
    for url in Playlist(url).video_urls[begin-1:end]:
        sleep(10) # HTTP 429 hack
        streams = YouTube(url).streams
        if streams.get_audio_only():
            default_filename = streams.get_audio_only().default_filename
            print(streams.get_audio_only().download())
            AudioSegment.from_file(default_filename).export(default_filename+'.wav', format='wav')
            wav_data, sr = sf.read(default_filename+'.wav', dtype=np.int16)
            assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
            waveform = wav_data / tf.int16.max#32768.0  # Convert to [-1.0, +1.0]
            waveform = waveform.astype('float32')

            # Convert to mono and the sample rate expected by YAMNet.
            if len(waveform.shape) > 1:
                waveform = np.mean(waveform, axis=1)
            if sr != params.sample_rate:
                waveform = resampy.resample(waveform, sr, params.sample_rate)

            waveforms.append(waveform)
            titles.append(YouTube(url).title)
        else:
            print('No audio_only stream!')
    return waveforms, titles

# data prep for TRAINING and TESTING

## extract features X and encode labels Y from classified playlists

In [ ]:
CLASSES = 2

In [ ]:
# TRAINING_data
playlists = [(0, ['https://www.youtube.com/playlist?list=PL0Q2eQA7p-wQIB3c6siLyc8qa25iIDiKq']), # Other
             (1, ['https://www.youtube.com/playlist?list=PL9deQcOOEJtV7_M52CH819jcN9Ty7ql0h'])] # 垃圾車

X = Y = 0
for label, playlist in playlists:
    waveforms, titles = data_from(*playlist)
    for waveform, title in zip(waveforms, titles):
        if type(X) == type(Y) == int:
            scores, embeddings, spectrogram = yamnet(waveform)
            X = embeddings
            Y = tf.one_hot(np.array([label] * len(embeddings)), depth=CLASSES)
        else:
            scores, embeddings, spectrogram = yamnet(waveform)
            X = np.concatenate((X, embeddings))
            Y = np.concatenate((Y, tf.one_hot(np.array([label] * len(embeddings)), depth=CLASSES)))
        print(X.shape, Y.shape, title)
TRAINING_data = X, Y

/content/目前最長的一期，只有五歲抬頭才能看懂的天能（信条）  老高與小茉 Mr & Mrs Gao.mp4
/content/圣诞歌曲 🎄  聖誕歌 英语 2020 🎅 聖誕歌曲合輯 英语 🎅 聖誕節歌曲 🎄 聖誕節 安靜音樂 🎅 祝大家聖誕快樂 🎄 🎅christmas songs.mp4
/content/City Street Ambience Sound Effect [FREE DOWNLOAD  ROYALTY FREE].mp4
/content/繁忙的街道和道路环境声 - 30分钟.mp4
/content/救護車的聲音.mp4
/content/警車警笛聲 EP1 提供大家下載！！！.mp4
/content/消防車聲精選.mp4
/content/小孩哭哭出來.mp4
/content/Natalie的嬰兒哭聲.mp4
/content/超級療癒的笑聲.mp4
(4516, 1024) (4516, 2) 目前最長的一期，只有五歲抬頭才能看懂的天能（信条） | 老高與小茉 Mr & Mrs Gao
(10043, 1024) (10043, 2) 圣诞歌曲 🎄  聖誕歌 英语 2020 🎅 聖誕歌曲合輯 英语 🎅 聖誕節歌曲 🎄 聖誕節 安靜音樂 🎅 祝大家聖誕快樂 🎄 🎅christmas songs
(10457, 1024) (10457, 2) City Street Ambience Sound Effect [FREE DOWNLOAD | ROYALTY FREE]
(14207, 1024) (14207, 2) 繁忙的街道和道路环境声 - 30分钟
(14269, 1024) (14269, 2) 救護車的聲音
(14351, 1024) (14351, 2) 警車警笛聲 EP.1 提供大家下載！！！
(14563, 1024) (14563, 2) 消防車聲精選
(14605, 1024) (14605, 2) 小孩哭哭出來
(14633, 1024) (14633, 2) Natalie的嬰兒哭聲
(14664, 1024) (14664, 2) 超級療癒的笑聲
/content/20200926彰化縣和美鎮垃圾車441-VQ，回收車4813-PU(切換音樂).mp4
/content/20200926和美鎮(新)

In [ ]:
# TESTING_data
playlists = [(0, ['https://www.youtube.com/playlist?list=PL0Q2eQA7p-wQwTP-Wk7nSHdpAFnU2kCSW']), # Other
             (1, ['https://www.youtube.com/playlist?list=PL9deQcOOEJtUKab8VBKnom0d8BrYBsvlg'])] # 垃圾車

X = Y = 0
for label, playlist in playlists:
    waveforms, titles = data_from(*playlist)
    for waveform, title in zip(waveforms, titles):
        if type(X) == type(Y) == int:
            scores, embeddings, spectrogram = yamnet(waveform)
            X = embeddings
            Y = tf.one_hot(np.array([label] * len(embeddings)), depth=CLASSES)
        else:
            scores, embeddings, spectrogram = yamnet(waveform)
            X = np.concatenate((X, embeddings))
            Y = np.concatenate((Y, tf.one_hot(np.array([label] * len(embeddings)), depth=CLASSES)))
        print(X.shape, Y.shape, title)
TESTING_data = X, Y

/content/Wydif after work party in Silicon Valley.mp4
/content/Apple - iPhone 5s - Dreams.mp4
/content/Welcome to Project Soli.mp4
/content/Annas Story.mp4
/content/Working at Google in Taiwan Software Engineering.mp4
/content/Life as Software Engineer @ Google Taiwan (Mandarin) 線上直播.mp4
/content/Official Intro  GTC 2019  I AM AI.mp4
/content/Making your Raspberry Pi 3B+ in Wales UK.mp4
/content/超恥辱! 公開我們的第一支影片! feat 頻道健檢活動.mp4
/content/CS50 Lecture by Mark Zuckerberg - 7 December 2005.mp4
(242, 1024) (242, 2) Wydif: after work party in Silicon Valley
(369, 1024) (369, 2) Apple - iPhone 5s - Dreams
(860, 1024) (860, 2) Welcome to Project Soli
(1096, 1024) (1096, 2) Anna's Story
(8568, 1024) (8568, 2) Working at Google in Taiwan: Software Engineering
(13301, 1024) (13301, 2) Life as Software Engineer @ Google Taiwan (Mandarin) 線上直播
(13646, 1024) (13646, 2) Official Intro | GTC 2019 | I AM AI
(13738, 1024) (13738, 2) Making your Raspberry Pi 3B+ in Wales, UK
(15833, 1024) (15833, 2) 超恥辱!

## know the shapes inside out

In [ ]:
STFTs = 1 # No matter how short wav is, even 0s, 1 STFT will be done!
SR = int(params.sample_rate)
scores, embeddings, spectrogram = yamnet(waveform[:round((0.015 + 0.48*STFTs + 0.001)*SR)]) # round rather than int to avoid 0.30000000000000004.com
print(scores.shape)
print(embeddings.shape)
print(spectrogram.shape)
assert scores.shape[1] == 521 # audio event classes
assert embeddings.shape[1] == 1024 # -dimension embedding
assert spectrogram.shape[1] == 64 # mel bins covering the range 125-7500 Hz
assert len(scores) == len(embeddings) == len(spectrogram) / 48 - 1 == STFTs

(1, 521)
(1, 1024)
(96, 64)


# model TRAINING: model.fit

In [ ]:
# https://www.tensorflow.org/guide/keras/sequential_model
model = keras.Sequential(
    [
        keras.Input(shape=1024),
        layers.Dense(512, activation="relu"),
        layers.Dense(2, activation="softmax"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 525,826
Trainable params: 525,826
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
X, Y = TRAINING_data
model.fit(x=X,
          y=Y,
          validation_split=0.2,
          batch_size=32,        # the smaller, the slower, limited by GPU if any
          epochs=100)

Epoch 1/100
499/499 [==============================] - 4s 6ms/step - loss: 0.0736 - accuracy: 0.9697 - val_loss: 0.2361 - val_accuracy: 0.9310
Epoch 2/100
499/499 [==============================] - 2s 5ms/step - loss: 0.0113 - accuracy: 0.9966 - val_loss: 0.8371 - val_accuracy: 0.7640
Epoch 3/100
499/499 [==============================] - 2s 5ms/step - loss: 0.0066 - accuracy: 0.9983 - val_loss: 0.6666 - val_accuracy: 0.8636
Epoch 4/100
499/499 [==============================] - 2s 5ms/step - loss: 0.0055 - accuracy: 0.9984 - val_loss: 0.5598 - val_accuracy: 0.8974
Epoch 5/100
499/499 [==============================] - 2s 5ms/step - loss: 0.0027 - accuracy: 0.9991 - val_loss: 0.6204 - val_accuracy: 0.9020
Epoch 6/100
499/499 [==============================] - 2s 5ms/step - loss: 5.6445e-04 - accuracy: 0.9999 - val_loss: 0.7527 - val_accuracy: 0.8889
Epoch 7/100
499/499 [==============================] - 2s 5ms/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 1.1641 - val_accuracy: 0.

# model TESTING: model.evaluate

In [ ]:
X, Y = TESTING_data
loss, accuracy = model.evaluate(X, Y)
accuracy

918/918 [==============================] - 2s 2ms/step - loss: 0.4539 - accuracy: 0.9734


0.9734100699424744

In [ ]:
model.save('model-126-1.h5')

# run inference w/wo a trained model on a playlist

## load a trained classifier

In [ ]:
#model = models.load_model('raspberrypi3-yamnet-sed/model-126-1.h5')

## test a playlist for 5MA inference

In [ ]:
playlist = ['https://www.youtube.com/playlist?list=PLk218Jsilveyr4dCO-LB-asqhIIl74Zt2']

MA = 5
SR = int(params.sample_rate)

waveforms, titles = data_from(*playlist)
for waveform, title in zip(waveforms, titles):
    #display(Audio(waveform, rate=SR)) # crashes if waveform is big
    print(title)
    #continue
    for i in range(MA, len(waveform)//SR, MA):
        scores, embeddings, spectrogram = yamnet(waveform[(i-MA)*SR:i*SR])
        classifier = np.argmax(np.mean(model.predict(embeddings[:-1]), axis=0), axis=-1)# = 0 as without a classifier
        if classifier == 0: # with only YAMNet
            # Scores is a matrix of (time_frames, num_classes) classifier scores.
            # Average them along time to get an overall classifier output for the clip.
            prediction = np.mean(scores[:-1], axis=0) # last score comes from insufficient samples
            # Report the highest-scoring classes #and their scores.
            top3 = np.argsort(prediction)[::-1][:3]
            print(f'{i//60}:{i%60:2d}',
                ''.join(f" {int(prediction[i]*10)}👈{yamnet_classes[i][:14].ljust(14, '　')}" for i in top3))
        elif classifier == 1:
            print(f'{i//60}:{i%60:2d}', "台灣音樂垃圾車")

/content/東河鄉公所 清潔隊 垃圾車 村莊 垃圾收運 (高畫質不專業錄影) 詳細請看說明.mp4
/content/東河鄉公所 清潔隊 垃圾車  村莊 垃圾收運 (高畫質不專業錄影) 詳細請看說明.mp4
/content/DONGFENG 裕隆東風 垃圾車 車內 視角 (高畫質不專業錄影) 2019年5月25拍攝.mp4
/content/DAF 達富 垃圾車 (高畫質不專業錄影) 詳細請看說明.mp4
東河鄉公所 清潔隊 垃圾車 村莊 垃圾收運 (高畫質不專業錄影) 詳細請看說明
0: 5 台灣音樂垃圾車
0:10 台灣音樂垃圾車
0:15 台灣音樂垃圾車
0:20 台灣音樂垃圾車
0:25 台灣音樂垃圾車
0:30 台灣音樂垃圾車
0:35 台灣音樂垃圾車
0:40 台灣音樂垃圾車
0:45 台灣音樂垃圾車
0:50 台灣音樂垃圾車
0:55 台灣音樂垃圾車
1: 0 台灣音樂垃圾車
1: 5 台灣音樂垃圾車
1:10 台灣音樂垃圾車
1:15 台灣音樂垃圾車
1:20 台灣音樂垃圾車
1:25 台灣音樂垃圾車
1:30 台灣音樂垃圾車
1:35 台灣音樂垃圾車
1:40 台灣音樂垃圾車
1:45 台灣音樂垃圾車
1:50 台灣音樂垃圾車
1:55 台灣音樂垃圾車
2: 0 台灣音樂垃圾車
2: 5 台灣音樂垃圾車
2:10 台灣音樂垃圾車
2:15 台灣音樂垃圾車
2:20 台灣音樂垃圾車
2:25 台灣音樂垃圾車
2:30 台灣音樂垃圾車
2:35 台灣音樂垃圾車
2:40 台灣音樂垃圾車
2:45 台灣音樂垃圾車
2:50 台灣音樂垃圾車
2:55 台灣音樂垃圾車
3: 0 台灣音樂垃圾車
3: 5 台灣音樂垃圾車
3:10 台灣音樂垃圾車
3:15 台灣音樂垃圾車
3:20 台灣音樂垃圾車
3:25 台灣音樂垃圾車
3:30 台灣音樂垃圾車
3:35 台灣音樂垃圾車
3:40 台灣音樂垃圾車
3:45 台灣音樂垃圾車
3:50 台灣音樂垃圾車
3:55 台灣音樂垃圾車
4: 0 台灣音樂垃圾車
4: 5 台灣音樂垃圾車
4:10 台灣音樂垃圾車
4:15 台灣音樂垃圾車
4:20 台灣音樂垃圾車
4:25 台灣音樂垃圾車
4:30 台灣音樂垃圾車
4:35 台灣音樂垃圾車
4:40 台灣音樂垃圾車
4:45 台灣音樂垃圾車
4:50 台灣音樂垃

# TBD: duplicate the last layer as a new model

In [ ]:
# feature_extractor = keras.Model(
#     inputs=yamnet.inputs,
#     outputs=[layer.output for layer in yamnet.layers],
# )